### Aims:

The aim is here to get the outline contour of a shell from an image (jpg or png usually). 

To do this we are using the Marching Squares Method from skimage.

If there are multiple shells (or shell views) in one image, then we can specify which shell we're after by using "right" or "left" in the position parameter of the the function __get_outline_contour__. If there's only one image, you could shorten the processing time by writing "longest" - this will simply give you the longest contour found in the image, which tends to be the outline contour we are looking for.

It is possible that due to certain patterns on the shell (or other circumstances) that the longest contours are not simply of the outline of the shell, but they might feed into the shell too. For these pots, it might be worth editing the image in some way, before finding the contours. To solve this, we'll create a function called __binarize_shell__. This function will work well with images with very dark backgrounds. It will turn the background white, whilst turning everything else black, with the hope that shell will then be better seen.

#### Imports

In [8]:
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import threshold_otsu
from skimage import measure
import os
import itertools as it
import csv
from skimage import data, img_as_float,io
from copy import deepcopy
from skimage.transform import rescale, resize

#### Functions

In [9]:
def get_outline_contour(img,position):
    # Input: grey-scaled image of shell, position of shell in image e.g. left,right, or "longest" for longest contour. 
    # Output: contour of desired shell.
    # Note: if there's only one shell in the image, use position="longest".
    
    # Get all contours:
    thresh = threshold_otsu(img)
    binary = img > thresh
    cont = measure.find_contours(binary, 0.8)
    
    # Compute lengths of contours:
    cont_ln = []
    for n, contour in enumerate(cont):
        cont_ln.append(len(contour))
        
    if position == "longest":
        k = np.argmax(cont_ln)
        contour = cont[k]
    
    else:
        # Find longest contours:
        longest_c = sorted(cont_ln,reverse=True)[:2] # Assuming there are no more than 2 shell views in one image
        long_ind = []
        minx = []
        for i in range(0,len(cont_ln)):
            if cont_ln[i] in longest_c:
                long_ind.append(i)
                c = cont[i]
                x = c[:,1]
                minx.append(min(x))

        if "left" in position:
            contour = cont[long_ind[np.argmin(minx)]]
        else:
            # if we're looking for the right-most contour
            contour = cont[long_ind[np.argmax(minx)]]        
            
    return contour

def binarize_shell(img):
    # Input: coloured image
    # Output: edited image that has either black or white pixels.
    
    # 1) Get image size
    ###################
    x_len = len(img[0])
    y_len = len(img)
    img2 = deepcopy(img)

    # 2) Binarize image
    ###################
    # Pixels that are not white will be turned black, all other pixels will be turned white.
    for i in range(0,y_len):
        for j in range(0,x_len):
            if sum(img[i][j]) <= 755:
                img2[i][j] = [0,0,0]
            else:
                img2[i][j] = [255,255,255]


    return img2

#### Automating and Saving Contours

In [10]:
# 1) Speciy folders:
img_pth = "backup/img/" # this is where the original images are located.
b_img = "backup/img_s/"

# 2) Iterate through shells in folder:
for shell in os.listdir(img_pth):
    # 2.1. Load grey-scaled image
    print(shell)
    if shell != ".DS_Store":   
        image = io.imread(img_pth+shell,as_gray=False) # Here, we load a grey-scaled version of the image.
        if image.shape[0] > 800:
            image = np.round(
                    rescale(image, 800 / (image.shape[0] * 1.0), preserve_range=True, multichannel=True)
            ).astype('uint8')
        elif image.shape[1] > 800:
            image = np.round(
                    rescale(image, 800 / (image.shape[1] * 1.0), preserve_range=True, multichannel=True)
            ).astype('uint8')

        io.imsave(b_img+"/"+shell, image)  # We temporarily save image so we can load it again as a grey-scaled image.

GULB15_0.jpg
BM65.jpg
NMS03_3.jpg
BM59.jpg
IPOT19_0.JPG
KOC49_1.jpg
VA62_1.jpg
VA01_2.jpg
HAM10.jpg
LACMA05_1.jpg
IPOT33.JPG
IPOT27.JPG
KOC74_1.jpg
KOC52.jpg
IPOT24_0.JPG
IPOT02_3.jpg
KOC46.jpg
VA55.jpg
VA41.jpg
IPOT26_2.JPG
KOC33_3.jpg
KOC33_2.jpg
IPOT26_3.JPG
VA54.jpg
DCC01.jpg
MET46.jpg
KOC47.jpg
IPOT02_2.jpg
KOC74_0.jpg
IPOT24_1.JPG
KOC53.jpg
IPOT32.jpg
LACMA05_0.jpg
VA01_3.jpg
VA62_0.jpg
IPOT19_1.JPG
KOC49_0.jpg
AM09.jpg
NMS03_2.jpg
BM64.jpg
GULB15_1.jpg
GULB15_3.jpg
NMS03_0.jpg
KOC49_2.jpg
IPOT19_3.JPG
GULB08.jpg
VA01_1.jpg
VA62_2.jpg
LACMA05_2.jpg
HAM07.jpg
KOC79.jpg
IPOT30.JPG
IPOT02_0.jpg
KOC45.jpg
KOC51.jpg
IPOT24_3.JPG
KOC74_2.jpg
MET44.jpg
LACMA09.jpg
VA56.jpg
IPOT26_1.JPG
KOC33_0.jpg
KOC33_1.jpg
IPOT26_0.JPG
VA57.jpg
LACMA08.jpg
MET45.jpg
IPOT24_2.JPG
KOC50.jpg
KOC74_3.jpg
KOC44.jpg
IPOT02_1.jpg
IPOT31.jpg
IPOT25.JPG
KOC78.jpg
LACMA05_3.jpg
VA62_3.jpg
VA01_0.jpg
GULB09.jpg
KOC49_3.jpg
IPOT19_2.JPG
NMS03_1.jpg
GULB15_2.jpg
BM63.jpg
MK08.jpg
VA59_1.jpg
IPOT04_3.jpg
KOC54_2.j

In [6]:
os.listdir(img_pth)[447]

'GULB17_0003_t.jpg'